In [1]:
import pandas as pd
import docker
import os
import re
import json
import matplotlib.pyplot as plt

'\nresult = pd.read_csv(\'analysis_result.csv\')\nans = result[\'ans\']\naddress = "../data_sol/"\n#tools = [\'mythril\',\'osiris\',\'slither\']\n#tools = [\'oyente\']\n#temp = "{mythril,osiris,slither}"\n#print(temp)\ntools = [\'oyente\']\nfor tool in tools:\n    for i in range(227,len(ans)):\n    #for i in range(5):\n        text = os.popen("cd ~/smartbugs;python3 smartBugs.py --tool {} --file {}".format(tool,address+ans[i])).read()\n        print(str(i))\n        print(text)\n        if(\'ERROR\' in text):\n            print(\'error\')'

In [3]:
#这个本质是根据一个特定文件夹的检测结果，然后存到json里边

In [4]:
#将结果map到dasp
def map_to(name,mapppp,head):
    mapping = open('mapping.json').read()
    mapp = json.loads(mapping)
    dic = dict(zip(head,[0]*10))
    for i in mapppp:
        #print(mapp['osiris'][i.strip()])
        if mapp[name][i.strip()] in dic.keys():
            dic[mapp[name][i.strip()]] += mapppp[i]
    return dic  
#返回dasp的keys
def get_head():
    address = './mapping/smartbugs.wiki/Vulnerabilities-mapping.md'
    f = open(address)
    m = {}
    asd = []
    temp = f.read().splitlines()
    head = temp[0].split('|')[3:-1]
    return head
#返回ans->json地址，code->合约名字
def get_addr(tool,root):
    ans = []
    code = []
    #root = './smartbugs/results' 
    #root = './smartbugs_result_contract' 
    directory = os.listdir(os.path.join(root,tool))
    print("directory {}".format(directory))
    for addr in directory:
        #if '20210501' in addr:#注意
        temp = os.path.join(root,tool,addr)
        #print(temp)
        code_dir = os.listdir(temp)#获得到达result.json的地址 
        #print(addr + ':'+str(code_dir))
        for code_addr in code_dir:
            result_addr = os.path.join(temp,code_addr,'result.json')#log.json
            ans.append(result_addr)
            code.append(code_addr)
    return ans,code
def barr(keys,values):
    fig = plt.figure(figsize=(12,4))
    plt.bar(keys, values)
    plt.xticks(rotation=-25) 
    plt.show()
def show_setno(addr):
    f = open(addr)
    t = f.read()
    a = t.splitlines()
    for i in range(len(a)):
        print(str(i) + a[i])    

In [6]:
#mythril
sol_name = []
anaylsis_flag = []
result = []
success = 0
cfalse = 0
afalse = 0
total = 0
error = 0
right = 0
check = []
mappp = {}
#root = './smartbugs/results'
tools = ['mythril']
#tool = 'mythril'
result_addrs,code = get_addr("mythril",root)
#print(len(result_addrs))
for i in range(len(result_addrs)):
    code_addr = code[i]
    result_addr = result_addrs[i] 
    file = open(result_addr)
    t = json.loads(file.read())
    #print(result_addr)
    if(code_addr not in check):
        total += 1
        sol_name.append(code_addr)
        check.append(code_addr)
        #print(type(t))
        #print(result_addr)
        if(t != None and t['analysis'] != None):
            #print(type(t['analysis']['success'])
            if(t['analysis']['success'] == True):
                success += 1
                anaylsis_flag.append(True)
                result.append(t['analysis']['issues'])
                if(len(t['analysis']['issues']) != 0):
                    #print("-------------------------")
                    error += 1
                    zxc =t['analysis']['issues']
                    #print(zxc[0])
                    #print(result_addr)
                    for ttt in t['analysis']['issues']:
                        asd = ttt['title']
                        #print(asd+"|",end=' ')
                        if asd in mappp:
                            mappp[asd] += 1
                        else:
                            mappp[asd] = 0
                    print()
                else:
                    right += 1
                    #print(t)
            else:
                #print("-------------------------")
                #print('not issues')
                afalse += 1
                anaylsis_flag.append(False)
                result.append(None)

        else:
            #print("-------------------------")
            #print("none")
           # print(t)
            anaylsis_flag.append(False)
            result.append(None)
            cfalse += 1
print("total:{} analysis false:{} analysis success:{} result right:{} result error:{} ".format(total,cfalse+afalse,success,right,error))
#mappp
mythril = pd.DataFrame({
    'sol_name':sol_name,
    'anaylsis_flag':anaylsis_flag,
    'result':result
})
mythril.to_json(os.path.join(json_root,mythril_name))

directory ['.ipynb_checkpoints', '20211403_1453', '20211403_1450', '20211403_1446', '20211403_1452', '20211403_1444', '20211403_1449', '20211403_1447', '20211403_1448', '20211403_1424', '20211403_1431', '20211403_1422', '20211403_1425', '20211403_1423', '20211403_1433', '20211403_1445']




















total:66 analysis false:15 analysis success:51 result right:31 result error:20 


In [13]:
#osiris
sol_name = []
result = []
check2 = []
right2 = []
error2 = []
false2 = []
addr,code = get_addr('osiris',root)
for i in range(len(addr)):
    #print(addr[i])
    file = open(addr[i])
    code_addr = code[i]
    t = json.loads(file.read())
    if(code_addr not in check2):
        sol_name.append(code_addr)
        check2.append(code_addr)#len(check) == total
        if(t != None):
            if(t['analysis'] != None):
                result.append(t['analysis'])
                #print(t['analysis'])
                for temp in t['analysis']:
                    if(len(temp['errors']) > 0):
                        error2.append(t)
                        break
                if(t not in error2):
                    right2.append(t)
            else:
                result.append(None)
                false2.append(t)
        else:
            result.append(None)
            false2.append(t)
print("total:{} analysis false:{} analysis success:{} result right:{} result error:{}".format(len(check2),len(false2),len(error2)+len(right2),len(right2),len(error2)))
mapp = {}
for i in error2:
    for temp in i['analysis']:
        if len(temp['errors']) > 0:
            for tt in temp['errors']:
                asd = tt['message']
                #print(asd)
                if asd in mapp:
                    mapp[asd] += 1
                else:
                    mapp[asd] = 0
                    #print(mapp[asd])
mapp
osiris = pd.DataFrame({
    'sol_name':sol_name,
    'result':result
})
osiris.to_json(os.path.join(json_root,osiris_name))

directory ['20211403_1521', '20211403_1510', '20211403_1517', '.ipynb_checkpoints', '20211403_1520', '20211403_1530', '20211403_1514', '20211403_1537', '20211403_1519', '20211403_1543', '20211403_1531', '20211403_1536', '20211403_1518', '20211403_1532', '20211403_1538', '20211403_1535', '20211403_1512', '20211403_1515', '20211403_1540', '20211403_1541', '20211403_1513', '20211403_1511', '20211403_1525', '20211403_1509', '20211403_1545', '20211403_1542', '20211403_1516', '20211403_1544']
total:66 analysis false:0 analysis success:66 result right:44 result error:22


In [31]:
#slither
sol_name =[]
result=[]
check3 = []
right3 = []
error3 = []
false3 = []
mapppp = {}
addr,code = get_addr('slither',root)
for i in range(len(addr)):
    #print(addr[i])
    file = open(addr[i])
    code_addr = code[i]
    t = json.loads(file.read())
    if(code_addr not in check3):
        sol_name.append(code_addr)
        check3.append(t)#len(check) == total
        if(t != None and t['analysis'] != None):
            result.append(t['analysis'])
            for temp in t['analysis']:
                asd = temp['check']
                if temp['impact'] != 'Informational':
                    #print(asd)
                    if asd in mapppp:
                        mapppp[asd] += 1
                    else:
                        mapppp[asd] = 0
                    if t not in error3:
                        error3.append(t)
            if t not in error3:
                right3.append(t)
        else:
            result.append(None)
            false3.append(t)
print("total:{} analysis false:{} analysis success:{} result right:{} result error:{}".format(len(check3),len(false3),len(error3)+len(right3),len(right3),len(error3)))
#mapppp
slither = pd.DataFrame({
    'sol_name':sol_name,
    'result':result    
}).drop_duplicates('sol_name')
print(len(slither))
slither.to_json(os.path.join(json_root,slither_name))

directory ['.ipynb_checkpoints', '20211403_1419', '20211403_1429', '20211403_1422', '20211403_1430', '20211403_1421', '20211403_1420', '20211403_1428']
total:132 analysis false:0 analysis success:132 result right:64 result error:68
66


In [11]:
#temp = map_to('slither',mapppp,head)
#barr(temp.keys(),temp.values())
#temp


In [26]:
#oyente
sol_name =[]
result=[]
check3 = []
right3 = []
error3 = []
false3 = []
mapppp = {}
a = 5
addr,code = get_addr('oyente',root)
for i in range(len(addr)):
    #print(addr[i])
    file = open(addr[i])
    code_addr = code[i]
    t = json.loads(file.read())
    if(code_addr not in check3):
        sol_name.append(code_addr)
        check3.append(t)#len(check) == total
        if len(t['analysis']) != 0:
            ans_temp = []
            for temp in t['analysis']:
                asd = temp['errors']
                name = temp['name']
                for qw in asd:
                    ans_temp.append({
                        'lineno':qw['line'],
                        'title':qw['message'],
                        'contract':name
                    })
                if t not in error3:
                        error3.append(t)
            if t not in error3:
                right3.append(t)
            result.append(ans_temp)
        else:
            result.append([])
            false3.append(t)
print("total:{} analysis false:{} analysis success:{} result right:{} result error:{}".format(len(check3),len(false3),len(error3)+len(right3),len(right3),len(error3)))
#mapppp
oyente = pd.DataFrame({
    'sol_name':sol_name,
    'result':result    
})
oyente = oyente.drop_duplicates(['sol_name'])
oyente[oyente['sol_name']!='simple_dao']
oyente.to_json(os.path.join(json_root,oyente_name))
#oyente.to_json(oyente_name)

directory ['20211403_1456', '20211403_1505', '20211403_1457', '20211403_1453', '20211403_1504', '20211403_1501', '20211403_1458', '20211403_1508', '20211403_1455', '20211403_1500', '20211403_1507', '20211403_1503', '20211403_1509', '20211403_1502', '20211403_1506', '20211403_1454']
total:66 analysis false:20 analysis success:46 result right:0 result error:46


In [ ]:
#mapping 获得mapping.json
address = '.Vulnerabilities-mapping.md'
f = open(address)
m = {}
asd = []
temp = f.read().splitlines()
head = temp[0].split('|')[3:-1]
target = ['mythril','oyente','slither','osiris']
for i in target:
    t = i.strip()
    m[t] = {}
for i in temp:
    if i.split()[0] in target:
        asd.append(i)
for i in head:
    m[i] = []
for i in asd:
    a = i.split('TRUE')[0].count('|') - 2
    name = i.split("|")[0].strip()
    vul = i.split("|")[1].strip()
    m[name][vul] = head[a]
    #m[head[a]].append(name)
asdd = json.dumps(m)
qwe = open('mapping.json','w')
qwe.write(asdd)
qwe.close()

# 没有mapping的版本